# Import des librairies 

In [1]:
import pandas as pd 
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import csv
import numpy as np
import re
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from concurrent import futures
import dateparser
import operator

import warnings
warnings.filterwarnings('ignore')

data_path="Data/"#Chemin du dossier pour enregistrer le fichier csv

In [2]:
def check_nan(df):
    for i in df.columns.tolist():
        print("Valeurs nan dans "+str(i)+" : "+str(df[i].isna().sum()))
        
def check_unique(df):
    for i in df.columns.tolist():
        print("Valeurs uniques dans "+str(i)+" : "+str(df[i].nunique()))

# Chargement des données déjà existantes

In [3]:
%%time
allo_titre = pd.read_csv(data_path+"allocine_titre_id.csv")
allo_home =pd.read_csv(data_path+"allocine_home_cleaned.csv")
allo_cast = pd.read_csv(data_path+"allocine_casting_movies.csv")
allo_id_cast =pd.read_csv(data_path+"allocine_id_people.csv")
allo_reviews = pd.read_csv(data_path+"allocine_reviews_cleaned.csv")

sc_rates=pd.read_csv(data_path+"Senscritique_ids_notes_cleaned.csv")

imdb_match =pd.read_csv(data_path+"IMDB_best_matching_ids_cleaned.csv") 
imdb_bo_lg = pd.read_csv(data_path+"IMDB_box_office_langues_cleaned.csv")

CPU times: total: 25 s
Wall time: 25.1 s


In [4]:
print("Shape Allociné Home:",allo_home.shape)
print("Shape Allociné Cast:",allo_cast.shape)
print("Shape Allociné id Cast:",allo_id_cast.shape)
print("Shape Allociné reviews:",allo_reviews.shape)
print("Shape Senscritique rates:",sc_rates.shape)
print("Shape IMDB matches:",imdb_match.shape)
print("Shape IMDB Box Office et Langues:",imdb_bo_lg.shape)

Shape Allociné Home: (19882, 9)
Shape Allociné Cast: (19982, 4)
Shape Allociné id Cast: (78448, 2)
Shape Allociné reviews: (923062, 6)
Shape Senscritique rates: (18344, 6)
Shape IMDB matches: (16929, 10)
Shape IMDB Box Office et Langues: (16929, 8)


# Jointures

## df = Home x Cast

In [5]:
allo_home.head()

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3"
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9"
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9"
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9"
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9"


In [6]:
allo_cast.head()

,movie_id,realisateurs,scenaristes,acteurs
0,178014,['1066'],"['49984', '1066', '1066', '49985', '49984', '4...","['41339', '34515', '259', '6407', '12973', '22..."
1,281293,['142452'],['142452'],"['41623', '21353', '101150', '646346', '60312'..."
2,289305,['16478'],"['267165', '923099', '267165', '267165', '1647...","['18273', '194479', '465360', '895145', '84779..."
3,266320,['692670'],"['575327', '575327', '97569']","['502680', '844314', '794353', '982500', '7069..."
4,288544,['507431'],"['507431', '923535']","['415513', '67071', '17614', '167254', '136213..."


In [7]:
df =pd.merge(allo_home, allo_cast, left_on="id",right_on="movie_id").drop(columns=["movie_id"])

## df = df x Reviews Allociné

In [8]:
allo_reviews.head()

,id_movie,pseudo,note,date,contenu_complet,contenu_sans_spoils
0,231865,AZZZO,"4,5",2018-04-27,\nLa mort d'un fils. Samuel Maoz part de ce th...,\nLa mort d'un fils. Samuel Maoz part de ce th...
1,231865,traversay1,"4,5",2018-04-27,\nDans cette vieille danse surannée qu'est le ...,\nDans cette vieille danse surannée qu'est le ...
2,231865,vidalger,"4,5",2018-04-28,"\nAprès Lebanon, chef d'œuvre absolu sur la gu...","\nAprès Lebanon, chef d'œuvre absolu sur la gu..."
3,231865,velocio,"4,0",2018-04-14,"\nIl y a 8 ans, le réalisateur israélien Samue...","\nIl y a 8 ans, le réalisateur israélien Samue..."
4,231865,Daniel C.,"5,0",2018-04-28,\nQue signifie mourir ? Comment supporter la p...,\nQue signifie mourir ? Comment supporter la p...


In [9]:
temp1 =allo_reviews.groupby('id_movie')['contenu_complet'].apply(list).reset_index()
temp2 = allo_reviews.groupby('id_movie')['contenu_sans_spoils'].apply(list).reset_index()
temp3 = pd.merge(temp1, temp2, left_on="id_movie", right_on="id_movie")
temp3.head()

,id_movie,contenu_complet,contenu_sans_spoils
0,1,[\nLa prestation sans faute de Jack Nicholson ...,[\nLa prestation sans faute de Jack Nicholson ...
1,2,"[\nMoi je l'aime ce film, je l'avais vu il y a...","[\nMoi je l'aime ce film, je l'avais vu il y a..."
2,3,[\nUne fantaisie ètourdissante des annèes 50 a...,[\nUne fantaisie ètourdissante des annèes 50 a...
3,4,"[\nCe sera une constante chez ce réalisateur, ...","[\nCe sera une constante chez ce réalisateur, ..."
4,5,"[\nComédie brillante sur la guerre des sexes,e...","[\nComédie brillante sur la guerre des sexes,e..."


In [10]:
df = pd.merge(df, temp3, left_on="id", right_on="id_movie", how="outer").drop(columns="id_movie").rename(columns={"contenu_complet":"commentaires_allocine",
                                                                                                                  "contenu_sans_spoils":"commentaires_sans_spoil_allocine",
                                                                                                                 "note_moyenne_spectateurs":"note_moyenne_spectateurs_allocine"})

## df = df x Senscritique

In [18]:
sc_rates= sc_rates.rename(columns={"id":"movie_id"}).drop(columns=["title"])
sc_rates["note_moyenne_spectateurs_sc"] = sc_rates["sc_rating"]/2

In [21]:
sc_rates.head()

,movie_id,sc_id,sc_rating,sc_detailed_similarity,score_mean,note_moyenne_spectateurs_sc
0,178014,avatar_la_voie_de_l_eau/450676,6.9,"(0.96, 1.0, 1.0, 0.9866666666666667)",0.986667,3.45
1,281293,les_banshees_d_inisherin/41368328,7.3,"(1.0, 1.0, 1.0, 1.0)",1.000000,3.65
2,289305,tempete/44158578,6.2,"(1.0, 1.0, 1.0, 1.0)",1.000000,3.10
3,266320,m3gan/44804275,5.4,"(1.0, 0.8, 1.0, 0.9333333333333332)",0.933333,2.70
4,288544,le_tourbillon_de_la_vie/47794530,6.2,"(1.0, 0.8, 1.0, 0.9333333333333332)",0.933333,3.10


In [23]:
df= pd.merge(df, sc_rates, left_on="id", right_on="movie_id", how="outer").drop(columns="movie_id") 

## df = df x IMBD matches

In [24]:
imdb_match.columns= ["id_movie","titre_imdb","id_imdb","year_imdb","actors_imdb","score_title_imdb","score_year_imdb",
                       "score_actors_imdb","score_mean_imdb","href_imdb"]
imdb_match.head()

,id_movie,titre_imdb,id_imdb,year_imdb,actors_imdb,score_title_imdb,score_year_imdb,score_actors_imdb,score_mean_imdb,href_imdb
0,249092,Blame!,tt6574146,2017.0,"['takahiro sakurai', 'kana hanazawa']",1.0,1.0,1.0,1.0,/title/tt6574146/?ref_=fn_tt_tt_1
1,178979,Kick-Ass 2,tt1650554,2013.0,"['aaron taylor-johnson', 'chloë grace moretz']",1.0,1.0,1.0,1.0,/title/tt1650554/?ref_=fn_tt_tt_1
2,129557,Pour elle,tt1217637,2008.0,"['vincent lindon', 'diane kruger']",1.0,1.0,1.0,1.0,/title/tt1217637/?ref_=fn_tt_tt_1
3,36464,Flic Story,tt0072996,1975.0,"['alain delon', 'jean-louis trintignant']",1.0,1.0,1.0,1.0,/title/tt0072996/?ref_=fn_tt_tt_1
4,33913,Dernier domicile connu,tt0064225,1970.0,"['lino ventura', 'marlène jobert']",1.0,1.0,1.0,1.0,/title/tt0064225/?ref_=fn_tt_tt_1


In [25]:
df= pd.merge(df, imdb_match, left_on="id", right_on="id_movie", how="outer").drop(columns="id_movie") 

## df = df x IMBD BoxOffice Langues

In [26]:
imdb_bo_lg.head()

,movie_id,href_imdb,pays_origine,langue_origine,budget,gross_domestic,opening_weekend_domestic,cumulative_world_wide_gross
0,249092,/title/tt6574146/?ref_=fn_tt_tt_1,['Japan'],['Japanese'],NaN,NaN,NaN,NaN
1,178979,/title/tt1650554/?ref_=fn_tt_tt_1,"['United Kingdom', 'United States', 'Japan']",['English'],28000000.0,28795985.0,13332955.0,60795985.0
2,129557,/title/tt1217637/?ref_=fn_tt_tt_1,"['France', 'Spain']","['French', 'Spanish']",7930000.0,NaN,NaN,6548942.0
3,36464,/title/tt0072996/?ref_=fn_tt_tt_1,"['France', 'Italy']",['French'],NaN,NaN,NaN,NaN
4,33913,/title/tt0064225/?ref_=fn_tt_tt_1,"['France', 'Italy']",['French'],NaN,NaN,NaN,NaN


In [27]:
df= pd.merge(df, imdb_bo_lg.rename(columns={"href_imdb":"href"}), left_on="id", right_on="movie_id", how="outer").drop(columns=["href","movie_id"])

# Observations 

In [28]:
print("Shape : ",df.shape)
check_nan(df)

df.head()

Shape :  (19882, 34)
Valeurs nan dans id : 0
Valeurs nan dans titre : 0
Valeurs nan dans date_sortie : 0
Valeurs nan dans support : 69
Valeurs nan dans duree : 202
Valeurs nan dans genres : 0
Valeurs nan dans synopsis : 280
Valeurs nan dans note_moyenne_presse : 10349
Valeurs nan dans note_moyenne_spectateurs_allocine : 113
Valeurs nan dans realisateurs : 148
Valeurs nan dans scenaristes : 703
Valeurs nan dans acteurs : 149
Valeurs nan dans commentaires_allocine : 824
Valeurs nan dans commentaires_sans_spoil_allocine : 824
Valeurs nan dans sc_id : 1538
Valeurs nan dans sc_rating : 2538
Valeurs nan dans sc_detailed_similarity : 1538
Valeurs nan dans score_mean : 1538
Valeurs nan dans note_moyenne_spectateurs_sc : 2538
Valeurs nan dans titre_imdb : 2953
Valeurs nan dans id_imdb : 2953
Valeurs nan dans year_imdb : 2966
Valeurs nan dans actors_imdb : 2953
Valeurs nan dans score_title_imdb : 2953
Valeurs nan dans score_year_imdb : 2953
Valeurs nan dans score_actors_imdb : 2953
Valeurs nan d

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs_allocine,realisateurs,...,score_year_imdb,score_actors_imdb,score_mean_imdb,href_imdb,pays_origine,langue_origine,budget,gross_domestic,opening_weekend_domestic,cumulative_world_wide_gross
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3",['1066'],...,1.0,1.0,0.986111,/title/tt1630029/?ref_=fn_tt_tt_1,['United States'],['English'],350000000.0,576316607.0,134100226.0,1.930117e+09
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9",['142452'],...,1.0,1.0,1.000000,/title/tt11813216/?ref_=fn_tt_tt_1,"['Ireland', 'United Kingdom', 'United States']",['English'],NaN,9319986.0,184454.0,2.778772e+07
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9",['16478'],...,1.0,1.0,1.000000,/title/tt21256038/?ref_=fn_tt_tt_1,['France'],['French'],12600000.0,NaN,NaN,4.801708e+06
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9",['692670'],...,1.0,1.0,1.000000,/title/tt8760708/?ref_=fn_tt_tt_1,"['United States', 'New Zealand']",['English'],12000000.0,62498045.0,30429860.0,9.760304e+07
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9",['507431'],...,1.0,1.0,1.000000,/title/tt13729364/?ref_=fn_tt_tt_1,['France'],['French'],NaN,NaN,NaN,1.380607e+06


# Sauvegarde données brutes 

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19882 entries, 0 to 19881
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 19882 non-null  int64  
 1   titre                              19882 non-null  object 
 2   date_sortie                        19882 non-null  object 
 3   support                            19813 non-null  object 
 4   duree                              19680 non-null  float64
 5   genres                             19882 non-null  object 
 6   synopsis                           19602 non-null  object 
 7   note_moyenne_presse                9533 non-null   object 
 8   note_moyenne_spectateurs_allocine  19769 non-null  object 
 9   realisateurs                       19734 non-null  object 
 10  scenaristes                        19179 non-null  object 
 11  acteurs                            19733 non-null  obj

In [30]:
df.to_csv(data_path+"all_data_brut.csv",sep=";",index=False)

# En ajoutant les noms du casting : 

In [31]:
df.iloc[:,:15].head()

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs_allocine,realisateurs,scenaristes,acteurs,commentaires_allocine,commentaires_sans_spoil_allocine,sc_id
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3",['1066'],"['49984', '1066', '1066', '49985', '49984', '4...","['41339', '34515', '259', '6407', '12973', '22...",[\nIl serait difficile de mettre moins de 3 ét...,[\nIl serait difficile de mettre moins de 3 ét...,avatar_la_voie_de_l_eau/450676
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9",['142452'],['142452'],"['41623', '21353', '101150', '646346', '60312'...","[\nAprès le triomphe de Three Billboards, voir...","[\nAprès le triomphe de Three Billboards, voir...",les_banshees_d_inisherin/41368328
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9",['16478'],"['267165', '923099', '267165', '267165', '1647...","['18273', '194479', '465360', '895145', '84779...","[\nVu en avant première, le film est émouvant,...","[\nVu en avant première, le film est émouvant,...",tempete/44158578
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9",['692670'],"['575327', '575327', '97569']","['502680', '844314', '794353', '982500', '7069...",[\nAvec un Chucky parti ensanglanter le petit ...,[\nAvec un Chucky parti ensanglanter le petit ...,m3gan/44804275
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9",['507431'],"['507431', '923535']","['415513', '67071', '17614', '167254', '136213...","[\nVoilà un beau film original, inédit, tel qu...","[\nVoilà un beau film original, inédit, tel qu...",le_tourbillon_de_la_vie/47794530


In [32]:
def convert_string_list(df, columns) : 
    for col in columns : 
        df[col] = df[col].apply(lambda list_ids : eval(list_ids) if list_ids!="nan" else np.nan)
    return df 

In [33]:
df["realisateurs"]= df["realisateurs"].astype(str)
df["scenaristes"] = df["scenaristes"].astype(str)
df["acteurs"] = df["acteurs"].astype(str)
df= convert_string_list(df, ["realisateurs","scenaristes","acteurs"])

## On remplace les ids par les vrais noms

In [36]:
%%time 
allo_id_cast["id"] = allo_id_cast["id"].astype(str)
dict_id_name = allo_id_cast.set_index("id")["nom"].to_dict()
for col in ['realisateurs','scenaristes', 'acteurs']: 
    df[col]= df[col].apply(lambda list_ids : operator.itemgetter(*list_ids)(dict_id_name) if type(list_ids)!=float else [] )
df.iloc[:,:15].head()

CPU times: total: 109 ms
Wall time: 115 ms


,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs_allocine,realisateurs,scenaristes,acteurs,commentaires_allocine,commentaires_sans_spoil_allocine,sc_id
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3",James Cameron,"(Amanda Silver, James Cameron, James Cameron, ...","(Sam Worthington, Zoe Saldana, Sigourney Weave...",[\nIl serait difficile de mettre moins de 3 ét...,[\nIl serait difficile de mettre moins de 3 ét...,avatar_la_voie_de_l_eau/450676
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9",Martin McDonagh,Martin McDonagh,"(Colin Farrell, Brendan Gleeson, Kerry Condon,...","[\nAprès le triomphe de Three Billboards, voir...","[\nAprès le triomphe de Three Billboards, voir...",les_banshees_d_inisherin/41368328
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9",Christian Duguay,"(Lilou Fogli, Christophe Donner, Lilou Fogli, ...","(Mélanie Laurent, Pio Marmaï, Kacey Mottet Kle...","[\nVu en avant première, le film est émouvant,...","[\nVu en avant première, le film est émouvant,...",tempete/44158578
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9",Gerard Johnstone,"(Akela Cooper, Akela Cooper, James Wan)","(Allison Williams, Violet McGraw, Ronny Chieng...",[\nAvec un Chucky parti ensanglanter le petit ...,[\nAvec un Chucky parti ensanglanter le petit ...,m3gan/44804275
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9",Olivier Treiner,"(Olivier Treiner, Camille Treiner)","(Lou de Laâge, Raphaël Personnaz, Isabelle Car...","[\nVoilà un beau film original, inédit, tel qu...","[\nVoilà un beau film original, inédit, tel qu...",le_tourbillon_de_la_vie/47794530


## Conversion en listes : 

In [37]:
for col in ['realisateurs','scenaristes', 'acteurs'] : 
    for i in range(len(df[col])): 
        if type(df[col].iloc[i])== str : 
            df[col].iloc[i]= [df[col].iloc[i]]
        else : 
            df[col].iloc[i] = list(df[col].iloc[i])
            

In [38]:
df.iloc[:,:15].head()

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs_allocine,realisateurs,scenaristes,acteurs,commentaires_allocine,commentaires_sans_spoil_allocine,sc_id
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3",[James Cameron],"[Amanda Silver, James Cameron, James Cameron, ...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[\nIl serait difficile de mettre moins de 3 ét...,[\nIl serait difficile de mettre moins de 3 ét...,avatar_la_voie_de_l_eau/450676
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9",[Martin McDonagh],[Martin McDonagh],"[Colin Farrell, Brendan Gleeson, Kerry Condon,...","[\nAprès le triomphe de Three Billboards, voir...","[\nAprès le triomphe de Three Billboards, voir...",les_banshees_d_inisherin/41368328
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9",[Christian Duguay],"[Lilou Fogli, Christophe Donner, Lilou Fogli, ...","[Mélanie Laurent, Pio Marmaï, Kacey Mottet Kle...","[\nVu en avant première, le film est émouvant,...","[\nVu en avant première, le film est émouvant,...",tempete/44158578
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9",[Gerard Johnstone],"[Akela Cooper, Akela Cooper, James Wan]","[Allison Williams, Violet McGraw, Ronny Chieng...",[\nAvec un Chucky parti ensanglanter le petit ...,[\nAvec un Chucky parti ensanglanter le petit ...,m3gan/44804275
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9",[Olivier Treiner],"[Olivier Treiner, Camille Treiner]","[Lou de Laâge, Raphaël Personnaz, Isabelle Car...","[\nVoilà un beau film original, inédit, tel qu...","[\nVoilà un beau film original, inédit, tel qu...",le_tourbillon_de_la_vie/47794530


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19882 entries, 0 to 19881
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 19882 non-null  int64  
 1   titre                              19882 non-null  object 
 2   date_sortie                        19882 non-null  object 
 3   support                            19813 non-null  object 
 4   duree                              19680 non-null  float64
 5   genres                             19882 non-null  object 
 6   synopsis                           19602 non-null  object 
 7   note_moyenne_presse                9533 non-null   object 
 8   note_moyenne_spectateurs_allocine  19769 non-null  object 
 9   realisateurs                       19882 non-null  object 
 10  scenaristes                        19882 non-null  object 
 11  acteurs                            19882 non-null  obj

In [40]:
df.to_csv(data_path+"all_data_brut_names.csv",sep=";",index=False)

# Data IA : 

In [41]:
df=pd.read_csv(data_path+"all_data_brut.csv", sep=";")
df.head(2)

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs_allocine,realisateurs,...,score_year_imdb,score_actors_imdb,score_mean_imdb,href_imdb,pays_origine,langue_origine,budget,gross_domestic,opening_weekend_domestic,cumulative_world_wide_gross
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3",['1066'],...,1.0,1.0,0.986111,/title/tt1630029/?ref_=fn_tt_tt_1,['United States'],['English'],350000000.0,576316607.0,134100226.0,1.930117e+09
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9",['142452'],...,1.0,1.0,1.000000,/title/tt11813216/?ref_=fn_tt_tt_1,"['Ireland', 'United Kingdom', 'United States']",['English'],NaN,9319986.0,184454.0,2.778772e+07


In [42]:
df.columns

Index(['id', 'titre', 'date_sortie', 'support', 'duree', 'genres', 'synopsis',
       'note_moyenne_presse', 'note_moyenne_spectateurs_allocine',
       'realisateurs', 'scenaristes', 'acteurs', 'commentaires_allocine',
       'commentaires_sans_spoil_allocine', 'sc_id', 'sc_rating',
       'sc_detailed_similarity', 'score_mean', 'note_moyenne_spectateurs_sc',
       'titre_imdb', 'id_imdb', 'year_imdb', 'actors_imdb', 'score_title_imdb',
       'score_year_imdb', 'score_actors_imdb', 'score_mean_imdb', 'href_imdb',
       'pays_origine', 'langue_origine', 'budget', 'gross_domestic',
       'opening_weekend_domestic', 'cumulative_world_wide_gross'],
      dtype='object')

In [43]:
df[["id","titre","date_sortie","support","duree","genres","realisateurs","scenaristes","acteurs","pays_origine","langue_origine",
   "budget","note_moyenne_presse","note_moyenne_spectateurs_allocine","note_moyenne_spectateurs_sc"]].to_csv(data_path+"data_brut_IA.csv",index=False, sep=";")